# Initialising Your Agent as a Transaction Endorser

This template walks you through the basic steps you need to take to configure your agent as an issuing authority on the Sovrin StagingNet. If using a different network you will need to update this template. The steps include:

* Writing your DID to the Sovrin StagingNet
* Accepting the Transaction Author Agreement

It is recommended that this initialisation notebook be run **once**. If you are following the default docker-compose services then your agents wallet storage will be persisted in a postgres database as long as you run `./manage stop` rather than `./manage down`. 



### Imports

In [1]:
from aries_cloudcontroller import AriesAgentController
import os
from termcolor import colored

### Initialise the Agent Controller

In [2]:
api_key = os.getenv("ACAPY_ADMIN_API_KEY")
admin_url = os.getenv("ADMIN_URL")

print(f"Initialising a controller with admin api at {admin_url} and an api key of {api_key}")
agent_controller = AriesAgentController(admin_url,api_key)

Initialising a controller with admin api at http://alice-agent:3021 and an api key of adminApiKey


## Start Webhook Server

In [3]:
webhook_port = int(os.getenv("WEBHOOK_PORT"))
webhook_host = "0.0.0.0"

await agent_controller.init_webhook_server(webhook_host, webhook_port)

print(f"Listening for webhooks from agent at http://{webhook_host}:{webhook_port}")

Listening webhooks failed! OSError(98, "error while attempting to bind on address ('0.0.0.0', 3010): address already in use") occurred.
Listening webhooks failed! OSError(98, "error while attempting to bind on address ('0.0.0.0', 3010): address already in use") occurred.


Exception: OSError(98, "error while attempting to bind on address ('0.0.0.0', 3010): address already in use")

In [ ]:
listeners = []

In [ ]:
def transaction_handler(payload):
    print("Transaction Handler")
    print(payload)
    
transaction_listener = {
    "topic": "transaction",
    "handler": transaction_handler

}

listeners.append(transaction_listener)

In [ ]:
# Receive connection messages
def connections_handler(payload):
    state = payload['state']
    connection_id = payload["connection_id"]
    their_role = payload["their_role"]
    routing_state = payload["routing_state"]
    
    print("----------------------------------------------------------")
    print("Connection Webhook Event Received")
    print("Connection ID : ", connection_id)
    print("State : ", state)
    print("Routing State : ", routing_state)
    print("Their Role : ", their_role)
    print("----------------------------------------------------------")
    if state == "active":
        # Your business logic
        print(colored("Connection ID: {0} is now active.".format(connection_id), "green", attrs=["bold"]))

connection_listener = {
    "handler": connections_handler,
    "topic": "connections"
}

listeners.append(connection_listener)

In [ ]:
agent_controller.register_listeners(listeners)

## Write DID to the Public Ledger

Note: if defined a ACAPY_WALLET_SEED value for your agent then this function will return a DID, but this DID still needs to be written to the ledger. If you did not define a seed you will need to create a DID first.

In [ ]:
public_did_response = await agent_controller.wallet.get_public_did()

In [ ]:
if public_did_response["result"]:
    did_obj = public_did_response["result"]
else:
    create_did_response = await agent_controller.wallet.create_did()
    did_obj = create_did_response['result']
print("DID", did_obj)

In [ ]:
# write new DID to Sovrin Stagingnet
import requests
import json 

url = 'https://selfserve.sovrin.org/nym'

payload = {"network":"stagingnet","did": did_obj["did"],"verkey":did_obj["verkey"],"paymentaddr":""}

# Adding empty header as parameters are being sent in payload
headers = {}

r = requests.post(url, data=json.dumps(payload), headers=headers)
print(r.json())

## Accept Transaction Author Agreement

Although the Sovrin StagingNet is permissionless, before DID's have the authority to write to the ledger they must accept something called a transaction author agreement by signing it using the DID they have on the ledger.

As a global public ledger, the Sovrin Ledger and all its participants are subject to privacy and data protection regulations such as the EU General Data Protection Regulation (GDPR). These regulations require that the participants be explicit about responsibilities for Personal Data.

To clarify these responsibilities and provide protection for all parties, the Sovrin Governance Framework Working Group developed an agreement between Transaction Authors and the Sovrin Foundation. The TAA can be found at Sovrin.org. It ensures that users are aware of and consent to the fact that all data written to the Sovrin Ledger cannot be removed, even if the original author of the transaction requests its removal.

The TAA outlines the policies that users must follow when interacting with the Sovrin Ledger. When a user’s client software is preparing a transaction for submission to the network, it must include a demonstration that the user had the opportunity to review the current TAA and accept it. This is done by including some additional fields in the ledger write transaction: 

* A hash of the agreement
* A date when the agreement was accepted, and
* A string indicating the user interaction that was followed to obtain the acceptance.

The Indy client API used by Sovrin has been extended to allow users to review current and past agreements and to indicate acceptance through an approved user interaction pattern. - source: https://sovrin.org/preparing-for-the-sovrin-transaction-author-agreement/

For more details on TAA please read more at the following links:
* [Preparing for the Sovrin Transaction Author Agreement](https://sovrin.org/preparing-for-the-sovrin-transaction-author-agreement/)
* [How the recent approval of the Sovrin Governance Framework v2 affects Transaction Authors
](https://sovrin.org/how-the-recent-approval-of-the-sovrin-governance-framework-v2-affects-transaction-authors/)
* [TAA v2](https://github.com/sovrin-foundation/sovrin/blob/master/TAA/TAA.md)
* [TAA Acceptance Mechanism List (AML)](https://github.com/sovrin-foundation/sovrin/blob/master/TAA/AML.md)

In [ ]:
taa_response = await agent_controller.ledger.get_taa()
TAA = taa_response['result']['taa_record']
TAA['mechanism'] = "service_agreement"
await agent_controller.ledger.accept_taa(TAA)

## Assign Agent Public DID if Not Set

Will only be ran if ACAPY_WALLET_SEED not initially set.

In [ ]:
if did_obj["posture"] != "public":
    response = await agent_controller.wallet.assign_public_did(did_obj["did"])
print("Successfully intialised agent with Public DID : ", did_obj["did"])

## Copy DID and Verkey from Author notebook

The endorser then writes this DID to the ledger

In [ ]:
did ="<author did>"
verkey="<author verykey>"
# Can be Trustee, Endorser etc.
role=None
alias="Aries Playground"

response = await agent_controller.ledger.register_nym(did, verkey, role, alias)

In [ ]:
print(response)

## Connect with Transaction Authors

## Create Invitation

Note the current arguments specified are in their default configurations. 

In [4]:
# Alias for invited connection
alias = None
auto_accept = False
# Use public DID?
public = "false"
# Should this invitation be usable by multiple invitees?
multi_use = "false"

invitation_response = await agent_controller.connections.create_invitation(alias, auto_accept, public, multi_use)
# Is equivalent to above. Arguments are optionally
# invitation_response = await agent_controller.connections.create_invitation()



# You probably want to keep this somewhere so you can enage in other protocols with this connection.
connection_id = invitation_response["connection_id"]


In [5]:
## Copy invitation to transaction author

In [6]:
print(invitation_response["invitation"])

{'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': '031ac045-25c8-4a3e-93fa-a352f263574f', 'recipientKeys': ['DskognYvM4eCDajkmHEGrCVe55GHSk9vBGGLMYzMrR7n'], 'serviceEndpoint': 'https://993b7f24cc22.ngrok.io', 'label': 'Alice'}


## Terminate Controller


In [ ]:
await agent_controller.terminate()